# Форматы данных (2)

Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

In [22]:
import csv
import json
import pickle
import pandas as pd
import numpy as np
import h5py

from bs4 import BeautifulSoup as bs
from pprint import pprint as pp


## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:
    
`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`

In [3]:
with open('./data/data/open_pubs.csv') as fp:
    reader = csv.reader(fp)
    header = next(reader)

    for row in reader:
        print(row)
        break

['fas_id', 'name', 'address', 'postcode', 'easting', 'northing', 'latitude', 'longitude', 'local_authority']


2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

In [9]:
A = np.random.randint(0, 100, size=(10_000, 10_000), dtype="int8")
B = np.random.randint(0, 100, size=(10_000, 10_000), dtype="int8")

In [10]:
np.save("A.npy", A)

In [7]:
A.dtype

dtype('int32')

In [16]:
np.savez("AB.npz", artem=A, nikita=B)

In [17]:
r = np.load("AB.npz")
r

In [18]:
r.files

['artem', 'nikita']

In [20]:
r['artem']  #r['arr_0']

array([[22, 47, 24, ..., 67, 47, 80],
       [90, 97, 58, ..., 49, 26, 20],
       [82, 95,  4, ..., 63, 16, 19],
       ...,
       [10,  0, 95, ..., 63, 86, 32],
       [15, 46, 59, ...,  8, 68, 18],
       [45, 53, 38, ..., 64, 77, 57]], dtype=int8)

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

In [26]:
with h5py.File('test.h5', 'w') as hdf:
    ds1 = hdf.create_dataset('arrA', data=A)
    ds2 = hdf.create_dataset('arrB', data=B)

    ds1.attrs["Description"] = "Здесь лежит массив А"
    ds2.attrs["Description"] = "Здесь лежит массив B"

In [28]:
with h5py.File('test.h5', 'r') as hdf:
    ds1 = hdf['arrA']
    print(type(ds1))
    arr = ds1[1500:5000]

<class 'h5py._hl.dataset.Dataset'>


In [29]:
arr

array([[ 4, 55, 20, ..., 12, 24,  9],
       [ 1, 35, 36, ..., 89, 95, 92],
       [61, 48,  1, ..., 44, 28, 16],
       ...,
       [43, 15, 12, ..., 17, 43, 42],
       [33, 69, 22, ..., 85, 28, 78],
       [62, 50, 16, ..., 13, 33, 96]], dtype=int8)

## Лабораторная работа 5

### csv

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [45]:
id_tags_dict = {}
with open("./data/data/tags_sample.csv") as fp:
    reader = csv.reader(fp)
    headers = next(reader)

    for row in reader:
        if row[0] not in id_tags_dict:
            id_tags_dict[row[0]] = list()
        id_tags_dict[row[0]].append(row[1])

with open("./data/data/tags_sample.json", 'w') as j:
    json.dump(id_tags_dict, j)

1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [46]:
# with open('../Lesson4-json_xml_pickle/data/data/recipes_sample_with_filled_nsteps.csv') as f:
#     reader = csv.reader(f)
#     header = next(reader)
#
#     for row in

pd.read_csv()

TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, треобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Сохраните результаты в файл `nutrition_grouped.h5`

3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.